In [1]:
import cv2
import os
import face_recognition

known_dir = 'known'
unknown_dir = 'unknown'
threshold = 70  # 임계값

# known 디렉토리에서 알려진 얼굴 이미지를 불러옵니다.
known_faces = []
known_names = []
for file_name in os.listdir(known_dir):
    if file_name.endswith('.jpg') or file_name.endswith('.jpeg') or file_name.endswith('.png'):
        image = face_recognition.load_image_file(os.path.join(known_dir, file_name))
        face_encoding = face_recognition.face_encodings(image)
        if len(face_encoding) > 0:
            known_faces.append(face_encoding[0])
            name = os.path.splitext(file_name)[0]  # 파일 이름에서 확장자를 제거하여 이름으로 사용합니다.
            known_names.append(name)

# unknown 디렉토리에서 알려진 얼굴 이미지를 불러옵니다.
unknown_faces = []
unknown_names = []
for file_name in os.listdir(unknown_dir):
    if file_name.endswith('.jpg') or file_name.endswith('.jpeg') or file_name.endswith('.png'):
        image = face_recognition.load_image_file(os.path.join(unknown_dir, file_name))
        face_encoding = face_recognition.face_encodings(image)
        if len(face_encoding) > 0:
            unknown_faces.append(face_encoding[0])
            name = os.path.splitext(file_name)[0]  # 파일 이름에서 확장자를 제거하여 이름으로 사용합니다.
            unknown_names.append(name)

video_capture = cv2.VideoCapture(0)

while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    # 프레임에서 얼굴 인식
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    for face_encoding, face_location in zip(face_encodings, face_locations):
        # known_faces와 비교하여 유사도 계산
        matches = face_recognition.face_distance(known_faces, face_encoding)
        min_distance = min(matches)
        min_distance_percent = (1 - min_distance) * 100

        if min_distance_percent > threshold:
            matched_index = matches.argmin()
            name = known_names[matched_index]
            (top, right, bottom, left) = face_location

            # 얼굴 부분에 사각형 표시 (초록색)
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            # 이름과 유사도 표시 (초록색)
            text = f"{name[:2]} ({min_distance_percent:.2f}%)"
            cv2.putText(frame, text, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            # Door Opened 표시 (초록색)
            cv2.putText(frame, "Door Opened", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        else:
            (top, right, bottom, left) = face_location

            matches = face_recognition.face_distance(unknown_faces, face_encoding)
            min_distance = min(matches)
            min_distance_percent = (1 - min_distance) * 100

            if min_distance_percent > threshold:
                matched_index = matches.argmin()
                name = unknown_names[matched_index]

                # 얼굴 부분에 사각형 표시 (파란색)
                cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 2)
                # 이름과 유사도 표시 (파란색)
                text = f"{name} ({min_distance_percent:.2f}%)"
                cv2.putText(frame, text, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
            else:
                # 얼굴 부분에 사각형 표시 (빨간색)
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                # Unknown 표시 (빨간색)
                cv2.putText(frame, "Unknown", (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                
                # unknown 이미지 저장
                unknown_image = frame[top:bottom, left:right]
                unknown_image_path = os.path.join(unknown_dir, f"unknown_{len(unknown_faces)}.jpg")
                cv2.imwrite(unknown_image_path, unknown_image)

                # 얼굴 이미지 저장
                face_image_path = os.path.join(unknown_dir, f"face_{len(unknown_faces)}.jpg")
                cv2.imwrite(face_image_path, frame)

                # unknown_faces와 unknown_names에 추가
                unknown_faces.append(face_encoding)
                unknown_names.append(f"unknown_{len(unknown_faces)}")

    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

